## Traitement de Données Massives

# Projet Partie n°1 

### Binôme : Sapy Oscar & Berthillon Mickaël

# Introduction

In [ ]:
#!bin/bash
# Dépendances
pip install colorthief piexif pillow autopep8 tesseract torch imageai tk pytesseract pyimage 
sudo apt install tkinter python3-tk tesseract-ocr-eng libtesseract-dev

# 0 : Importation des librairies

In [1]:
# Gérer les imports
import requests # pour les requêtes HTTP
import os # pour les chemins de fichiers
import json # pour les fichiers JSON
import cv2 # pour les images
import pytesseract # reconnaisance de texte
import torch
import random

import tkinter as tk # pour l'interface graphique de la décision

from PIL import Image, ImageTk # traiter les images
from PIL.ExifTags import TAGS # pour les métadonnées
from PIL.TiffImagePlugin import IFDRational


from colorthief import ColorThief # trouver la couleur dominante

from torchvision.models.detection import fasterrcnn_resnet50_fpn # trouver les objets / visages


# 1 : Collecte de données

In [ ]:
# image_downloader.py
#

# accès api pexels
url = "https://api.pexels.com/v1/search"
access_key = "MgxBjp4CnSFtTvWe3SeHtkTGp5oUOBWVG5S0PYZAd37hV3T3h2yY4PHp"

# header d'authentification basique
headers = {
    "Authorization": access_key
}

# construction de la requête
path = os.path.abspath("./images")
num_images = 15

query_params = {
    "query": "random",
    "per_page": num_images
}

# créée le dossier de destination si il n'existe pas
if not os.path.exists(path):
    os.makedirs(path)

# on veut savoir combien d'images il manque
existing_images = len([f for f in os.listdir(path) if f.endswith(('.jpg', '.jpeg', '.png'))])
missing_images = max(num_images - existing_images, 0)
##print(f"Images manquantes: {missing_images}")

# 
query_params["per_page"] = num_images

# envoi de la requête et téléchargement des images
if missing_images > 0:
    response = requests.get(url, headers=headers, params=query_params)
    json = response.json()
    photos = json.get("photos", [])

    downloaded = 0
    for photo in photos:
        if downloaded >= missing_images:
            break

        image_url = photo["src"]["original"]
        image_id = photo["id"]
        image_extension = ".jpg"
        image_filename = f"{image_id}{image_extension}"
        image_path = os.path.join(path, image_filename)

        if not os.path.exists(image_path):
            # 
            response = requests.get(image_url)
            with open(image_path, "wb") as f:
                f.write(response.content)
            downloaded += 1


# 2 : Etiquetage et annotation

In [5]:
# metadata_extractor.py
#

# encodeur json pour les bytes et les rationnels (pour les métadonnées)
class MyEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, bytes):
            try:
                return obj.decode('utf-8')
            except UnicodeDecodeError:
                return obj.decode('utf-8', 'replace')
        elif isinstance(obj, IFDRational):
            return float(obj)
        return json.JSONEncoder.default(self, obj)

# chemin vers le dossier images
path_to_folder = os.path.abspath('./images')

# crée le dossier metadata s'il n'xiste pas
if not os.path.exists('metadata'):
    os.mkdir('metadata')

# boucle sur les images du dossier
for filename in os.listdir(path_to_folder):
    if filename.endswith('.jpg') or filename.endswith('.jpeg') or filename.endswith('.png'):
        # extrait les métadonnées
        image = Image.open(os.path.join(path_to_folder, filename))
        exifdata = image.getexif()
        metadata = {}

        # ajoute les métadonnées à la liste
        for tag_id, value in exifdata.items():
            tag = TAGS.get(tag_id, tag_id)
            metadata[tag] = value

        # Convert the dictionary to JSON and store it in a file
        json_filename = os.path.splitext(filename)[0] + '.json'
        with open(os.path.join('metadata', json_filename), 'w') as f:
            json.dump(metadata, f, cls=MyEncoder)

/tmp/ipykernel_2754/627564814.py:27: ResourceWarning: unclosed file <_io.BufferedReader name='/home/mick/cour/tdm/new/Project-BigData/images/4226881.jpg'>
  image = Image.open(os.path.join(path_to_folder, filename))
/tmp/ipykernel_2754/627564814.py:27: ResourceWarning: unclosed file <_io.BufferedReader name='/home/mick/cour/tdm/new/Project-BigData/images/7004697.jpg'>
  image = Image.open(os.path.join(path_to_folder, filename))
/tmp/ipykernel_2754/627564814.py:27: ResourceWarning: unclosed file <_io.BufferedReader name='/home/mick/cour/tdm/new/Project-BigData/images/4022082.jpg'>
  image = Image.open(os.path.join(path_to_folder, filename))
/tmp/ipykernel_2754/627564814.py:27: ResourceWarning: unclosed file <_io.BufferedReader name='/home/mick/cour/tdm/new/Project-BigData/images/4386404.jpg'>
  image = Image.open(os.path.join(path_to_folder, filename))
/tmp/ipykernel_2754/627564814.py:27: ResourceWarning: unclosed file <_io.BufferedReader name='/home/mick/cour/tdm/new/Project-BigData/im

# 3 : Analyse de données

In [6]:
import os
import json
import torch
from PIL import Image
from PIL.ExifTags import TAGS
from transformers import ViTFeatureExtractor, ViTModel
import cv2
from colorthief import ColorThief
import pytesseract
from imageai.Detection import ObjectDetection

import torchvision.transforms as T
from torchvision.models.detection import fasterrcnn_resnet50_fpn

# initialise le modèle ViT en mode évaluation
model = fasterrcnn_resnet50_fpn(pretrained=True)
model.eval()

image_path = os.path.abspath("./images")

# fonction pour récupérer les métadonnées d'une image
def get_image_metadata(image_path):
    metadata = {}

    # lire l'image avec PIL et OpenCV
    pil_image = Image.open(image_path)
    cv_image = cv2.imread(image_path)

    # obtenir la couleur dominante et la palette de couleurs
    color_thief = ColorThief(image_path)
    dominant_color = color_thief.get_color(quality=1)
    color_palette = color_thief.get_palette(color_count=5)

    # l'orientation de l'image
    exif_data = pil_image._getexif()
    orientation = exif_data[274] if exif_data and 274 in exif_data else "unknown"

    # reconnaissance faciale
    face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")
    gray = cv2.cvtColor(cv_image, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.3, minNeighbors=3)
    num_faces = len(faces)

    # reconnaissance de texte
    pil_image_rgb = pil_image.convert('RGB')
    text = pytesseract.image_to_string(pil_image_rgb)

    # détection d'objets
    transform = T.Compose([T.Resize(256), T.CenterCrop(224), T.ToTensor()])
    img = transform(Image.open(image_path))
    img = img.unsqueeze(0)
    detections = model(img)

    objects = []
    for label, score in zip(detections[0]["labels"], detections[0]["scores"]):
        if score > 0.5:
            objects.append(label.item())


    # dict des métadonnées
    metadata = {
        "orientation": orientation,
        "dominant_color": dominant_color,
        "color_palette": color_palette,
        "num_faces": num_faces,
        "text": text,
        "objects": objects,
    }

    return metadata

# on boucle sur les images du dossier
for filename in os.listdir(path_to_folder):
    if filename.endswith(".jpg") or filename.endswith(".jpeg") or filename.endswith(".png"):
        # on récupère les métadonnées
        image_path = os.path.join(path_to_folder, filename)
        metadata = get_image_metadata(image_path)

        # on sauvegarde les métadonnées dans un fichier json
        with open(os.path.join("metadata", os.path.splitext(filename)[0] + ".json"), "w") as f:
            json.dump(metadata, f)

# 4 : Visualisation de données

In [ ]:
#

5

In [9]:
import json
import os
import tkinter as tk
from tkinter import filedialog
from PIL import Image, ImageTk

class ImageSelector(tk.Tk):
    def __init__(self, preferences_file, image_folder):
        super().__init__()

        self.preferences_file = preferences_file
        self.image_folder = image_folder
        self.images = os.listdir(image_folder)
        self.preferences = self.read_preferences()

        self.current_image_index = 0
        self.image_label = tk.Label(self)
        self.image_label.pack()

        self.like_button = tk.Button(self, text="J'aime", command=self.like_image)
        self.like_button.pack()

        self.next_button = tk.Button(self, text="Suivant", command=self.show_next_image)
        self.next_button.pack()

        self.quit_button = tk.Button(self, text="Quitter", command=self.quit_app)
        self.quit_button.pack()

        self.show_next_image()

    def read_preferences(self):
        if os.path.exists(self.preferences_file):
            with open(self.preferences_file, "r") as f:
                return json.load(f)
        else:
            return {}

    def write_preferences(self):
        with open(self.preferences_file, "w") as f:
            json.dump(self.preferences, f)

    def show_next_image(self):
        if self.current_image_index < len(self.images):
            image_file = self.images[self.current_image_index]
            image_path = os.path.join(self.image_folder, image_file)

            image = Image.open(image_path)
            image.thumbnail((400, 400))
            photo = ImageTk.PhotoImage(image)

            self.image_label.config(image=photo)
            self.image_label.image = photo

            self.current_image_index += 1
        else:
            self.quit_app()

    def like_image(self):
        image_file = self.images[self.current_image_index - 1]
        self.preferences[image_file] = True
        self.write_preferences()

    def quit_app(self):
        self.write_preferences()
        self.destroy()

preferences_file = "preferences.json"
image_folder = "images"
app = ImageSelector(preferences_file, image_folder)
app.mainloop()


NameError: name 'current_image_index' is not defined